### Needed libraries

In [ ]:
import rawpy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
# Set plot style
plt.style.use('seaborn-deep')

# Directory containing RAW images
raw_dir = Path('./')

# Output Excel file
output_excel = 'mean_rgb_values_center.xlsx'

# Create empty DataFrame to store RGB values
data = pd.DataFrame(columns=['Image', 'Red', 'Green', 'Blue'])

def process_raw_image(file_path: Path) -> tuple:
    """Reads a RAW image and returns mean RGB values from its center."""
    try:
        with rawpy.imread(str(file_path)) as raw:
            rgb = raw.postprocess(gamma=(1, 1), no_auto_bright=True, output_bps=8)
            h, w = rgb.shape[:2]
            center_crop = rgb[h//2 - 500:h//2 + 500, w//2 - 500:w//2 + 500]
            mean_r = np.mean(center_crop[:, :, 0]) / 1000
            mean_g = np.mean(center_crop[:, :, 1]) / 1000
            mean_b = np.mean(center_crop[:, :, 2]) / 1000
            return mean_r, mean_g, mean_b
    except Exception as e:
        print(f"Error processing {file_path.name}: {e}")
        return None, None, None

# Process images and collect RGB data
for i in range(1283, 1322):
    file_path = raw_dir / f'DSC_{i}.NEF'
    if file_path.exists():
        r, g, b = process_raw_image(file_path)
        if r is not None:
            data.loc[len(data)] = {'Image': i, 'Red': r, 'Green': g, 'Blue': b}
    else:
        print(f"File not found: {file_path}")
# Save RGB data to Excel
data.to_excel(output_excel, index=False)
print(f"Saved mean RGB values to {output_excel}")

# --- Plotting ---
# Load processed data
df = pd.read_excel('./mean_rgb_values_center.xlsx')

# Plot spectral response
df.plot(kind='line', x='Bands nm')
plt.title('Recovered Nikon D850 Spectral Sensitivity')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Relative Intensity')
plt.grid(True)
plt.tight_layout()
plt.show()